**Goal of the project**

The predictive response models used to help identify customers in marketing can also be used to help outbound sales teams improve their call conversion rate by targeting the best people or companies to call. Whether we’re sending emails or using catalogue marketing, or calling customers by phone, the principles are identical - we’re aiming to increase profit by generating the maximum amount of revenue from the minimum amount of effort and cost.

In this project, I’ll create a customer response model. It uses [IBM Watson Analytics data](https://www.kaggle.com/datasets/pankajjsh06/ibm-watson-marketing-customer-value-data) to predict if a customer will accept a renewal proposal when called.

**Load the packages**

In [274]:
# Importing libraries
import pandas as pd
import numpy as np

**Load the data**

In [275]:
# Load dataset
df = pd.read_csv('../input/ibm-watson-marketing-customer-value-data/WA_Fn-UseC_-Marketing-Customer-Value-Analysis.csv').iloc[: , 1:]

In [276]:
# Rename Pandas columns to lower case
df.columns = df.columns.str.lower()

In [277]:
df = df.applymap(lambda s: s.lower() if type(s) == str else s)

In [278]:
# Examine the data
df.head()

,state,customer lifetime value,response,coverage,education,effective to date,employmentstatus,gender,income,location code,...,months since policy inception,number of open complaints,number of policies,policy type,policy,renew offer type,sales channel,total claim amount,vehicle class,vehicle size
0,washington,2763.519279,no,basic,bachelor,2/24/11,employed,f,56274,suburban,...,5,0,1,corporate auto,corporate l3,offer1,agent,384.811147,two-door car,medsize
1,arizona,6979.535903,no,extended,bachelor,1/31/11,unemployed,f,0,suburban,...,42,0,8,personal auto,personal l3,offer3,agent,1131.464935,four-door car,medsize
2,nevada,12887.431650,no,premium,bachelor,2/19/11,employed,f,48767,suburban,...,38,0,2,personal auto,personal l3,offer1,agent,566.472247,two-door car,medsize
3,california,7645.861827,no,basic,bachelor,1/20/11,unemployed,m,0,suburban,...,65,0,7,corporate auto,corporate l2,offer1,call center,529.881344,suv,medsize
4,washington,2813.692575,no,basic,bachelor,2/3/11,employed,m,43836,rural,...,44,0,1,personal auto,personal l1,offer1,agent,138.130879,four-door car,medsize


In [279]:
# Overview of all variables, their datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   state                          9134 non-null   object 
 1   customer lifetime value        9134 non-null   float64
 2   response                       9134 non-null   object 
 3   coverage                       9134 non-null   object 
 4   education                      9134 non-null   object 
 5   effective to date              9134 non-null   object 
 6   employmentstatus               9134 non-null   object 
 7   gender                         9134 non-null   object 
 8   income                         9134 non-null   int64  
 9   location code                  9134 non-null   object 
 10  marital status                 9134 non-null   object 
 11  monthly premium auto           9134 non-null   int64  
 12  months since last claim        9134 non-null   i

**Define the target variable**

At the moment, the response column contains a boolean yes or no value, but we need to “binarise” this to turn it into a numeric value the model can use. A simple replace( ) is one of several ways to do this.

In [280]:
df['response'] = df['response'].replace(('yes', 'no'), (1, 0))

As we’ll see from examining the value_counts( ) of the target variable column, this dataset is imbalanced.

In [281]:
df['response'].value_counts()

0    7826
1    1308
Name: response, dtype: int64

**Changing a date string to a datetime**

In [282]:
df['effective to date'] = pd.to_datetime(df['effective to date'])

**Engineer datetime feature**

Basically we can break apart the effective to date and get the week.

In [283]:
df['week'] = df['effective to date'].dt.strftime('%W').astype(int)

In [284]:
df = df.drop('effective to date', axis = 1)

**Check for missing values**

Before moving on, we’ll check to see if there are any null values to impute. However, the data were all fine, so there was nothing to do.

In [285]:
df.isnull().sum()

state                            0
customer lifetime value          0
response                         0
coverage                         0
education                        0
employmentstatus                 0
gender                           0
income                           0
location code                    0
marital status                   0
monthly premium auto             0
months since last claim          0
months since policy inception    0
number of open complaints        0
number of policies               0
policy type                      0
policy                           0
renew offer type                 0
sales channel                    0
total claim amount               0
vehicle class                    0
vehicle size                     0
week                             0
dtype: int64

**Define X and y**

In [286]:
X = df.drop('response', axis = 1)

In [287]:
y = df['response']

**Create the training and test datasets**

To divide X and y into the train and test datasets we need to train the model we will use the train_test_split( ) function from scikit-learn. We’ll assign 30% of the data to the test groups using the argument test_size = 0.3, and we’ll use the stratify = y option to ensure the target variable is present in the test and train data in equal proportions.

In [288]:
from sklearn.model_selection import train_test_split

In [289]:
# Isolate X and y variables, and perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

**Create a pipeline**

First, we’re extracting the numeric columns from X and returning a list, then we’re doing the same with the categorical data columns. For the categorical data, we’re applying the OneHotEncoder( ) to ensure everything is numeric. These settings get passed to the ColumnTransformer( ).

Next, we’re using the pipeline from imblearn, which is specifically designed for working with imbalanced datasets like this one. The pipeline first runs our preprocessor we just created, then uses the SVM-SMOTE method to handle the class imbalance. Finally, we use the StandardScaler( ) to put our data on a consistent scale, then pass in the model and return a bundled_pipeline.

In [290]:
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SVMSMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [291]:
def get_pipeline(X, model):

    numeric_columns = list(X.select_dtypes(exclude = ['object']).columns.values.tolist())    
    categorical_columns = list(X.select_dtypes(include = ['object']).columns.values.tolist())
    categorical_pipeline = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
    
    preprocessor = ColumnTransformer(transformers = [('numeric', 'passthrough', numeric_columns),
                                                     ('categorical', categorical_pipeline, categorical_columns)], remainder = 'passthrough')

    bundled_pipeline = imbpipeline(steps = [('preprocessor', preprocessor),
                                            ('smote', SVMSMOTE(random_state = 42)),
                                            ('scaler', StandardScaler()),
                                            ('model', model)])
    
    return bundled_pipeline

**Select the best model for the job**

Rather than simply selecting a single model, or repeating our code manually on a range of models, we can create another function to automatically test a wide range of possible models to determine the best one for our needs.

In [292]:
from xgboost import XGBClassifier, XGBRFClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, HistGradientBoostingClassifier
from catboost import CatBoostClassifier

In [293]:
def select_model(X, y, pipeline = None):

  classifiers = {}
  classifiers.update({'XGBClassifier': XGBClassifier(random_state = 42)})
  classifiers.update({'XGBRFClassifier': XGBRFClassifier(random_state = 42)})
  classifiers.update({'LGBMClassifier': LGBMClassifier(random_state = 42)})
  classifiers.update({'DecisionTreeClassifier': DecisionTreeClassifier(random_state = 42)})
  classifiers.update({'RandomForestClassifier': RandomForestClassifier(random_state = 42)})
  classifiers.update({'ExtraTreesClassifier': ExtraTreesClassifier(random_state = 42)})
  classifiers.update({'GradientBoostingClassifier': GradientBoostingClassifier(random_state = 42)})    
  classifiers.update({'BaggingClassifier': BaggingClassifier(random_state = 42)})
  classifiers.update({'AdaBoostClassifier': AdaBoostClassifier(random_state = 42)})
  classifiers.update({'HistGradientBoostingClassifier': HistGradientBoostingClassifier(random_state = 42)})
  classifiers.update({'CatBoostClassifier': CatBoostClassifier(silent = True, random_state = 42)})

  df_models = pd.DataFrame(columns = ['model', 'run_time', 'f1_score_cv', 'f1_score'])

  for key in classifiers:

      print('*', key)

      start_time = time.time()
      
      pipeline = get_pipeline(X_train, classifiers[key])

      cv = cross_val_score(pipeline, X, y, cv = 5, scoring = 'f1_macro', n_jobs = -1)

      pipeline.fit(X_train, y_train)
      y_pred = pipeline.predict(X_test)

      row = {'model': key,
             'run_time': format(round((time.time() - start_time) / 60, 2)),
             'f1_score_cv': cv.mean(),
             'f1_score': f1_score(y_test, y_pred, average = 'macro')}

      df_models = df_models.append(row, ignore_index = True)

  df_models = df_models.sort_values(by = 'f1_score', ascending = False)
      
  return df_models

In [294]:
models = select_model(X_train, y_train)

* XGBClassifier
* XGBRFClassifier
* LGBMClassifier
* DecisionTreeClassifier
* RandomForestClassifier
* ExtraTreesClassifier
* GradientBoostingClassifier
* BaggingClassifier
* AdaBoostClassifier
* HistGradientBoostingClassifier
* CatBoostClassifier


Running the select_model( ) function on our training data takes a minute or so. The best independent model was BaggingClassifier. 

In [295]:
models.head(10)

,model,run_time,f1_score_cv,f1_score
10,CatBoostClassifier,1.51,0.967732,0.983010
0,XGBClassifier,0.32,0.970514,0.981013
5,ExtraTreesClassifier,0.15,0.957702,0.979625
4,RandomForestClassifier,0.16,0.947453,0.969381
2,LGBMClassifier,0.13,0.905936,0.928180
7,BaggingClassifier,0.12,0.940206,0.925641
9,HistGradientBoostingClassifier,0.14,0.909291,0.912114
3,DecisionTreeClassifier,0.07,0.886083,0.846914
6,GradientBoostingClassifier,0.29,0.649959,0.655731
8,AdaBoostClassifier,0.12,0.624645,0.626591


Finally, we can take our best model and fit the data on this.

In [296]:
selected_model = BaggingClassifier(random_state = 42)
bundled_pipeline = get_pipeline(X_train, selected_model)
bundled_pipeline.fit(X_train, y_train)
y_pred = bundled_pipeline.predict(X_test)

**Assess the performance of the model**

To examine how well the model performed in a little more detail we can make use of the classification_report( ) function. The classification report shows us the precision, recall, and F1 score for our predictions.

In [297]:
from sklearn.metrics import classification_report

In [298]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2348
           1       0.90      0.85      0.87       393

    accuracy                           0.96      2741
   macro avg       0.94      0.92      0.93      2741
weighted avg       0.96      0.96      0.96      2741

